In [ ]:
import os
import sys

lib_path = os.path.abspath(os.path.join('..'))
if lib_path not in sys.path:
    sys.path.append(lib_path)

In [ ]:
import pandas as pd
from lib.helper import load_coin_data, filter_by_years, split_train_test

In [ ]:
# load dataset
df = load_coin_data()

In [ ]:
# for now just work with 'bitcoin'
df = df[df['slug'] == 'bitcoin'].reset_index(drop=True)
df.info()

In [ ]:
df

In [ ]:
# we'll use three previous closing prices as the input variables. ('close', 'close-1', 'close-2')
df['close-1'] = df['close'].shift(1)
df['close-2'] = df['close'].shift(2)
df = df.dropna()


In [ ]:
df

In [ ]:
train, test = split_train_test(df)

X_train = train[['close', 'close-1', 'close-2']].values
X_test = test[['close', 'close-1', 'close-2']].values

y_train = train['next_close'].values
y_test = test['next_close'].values

In [ ]:
print(f'x_test shape: {X_test.shape}')
print(f'x_train shape: {X_train.shape}')
print(f'y_test shape: {y_test.shape}')
print(f'y_train shape: {y_train.shape}')

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def scale_data(data):
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)
    return scaler, data_scaled


X_train_scaler, X_train_scaled = scale_data(X_train)
X_test_scaler, X_test_scaled = scale_data(X_test)

y_train_reshaped = y_train.reshape(-1, 1)
y_test_reshaped = y_test.reshape(-1, 1)

y_train_scaler, y_train_scaled = scale_data(y_train_reshaped)
y_test_scaler, y_test_scaled = scale_data(y_test_reshaped)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(3, input_dim=3, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=10,
    batch_size=32)